In [13]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
from scipy.optimize import minimize_scalar

from tqdm.contrib.telegram import tqdm

In [14]:
## get problem parameters and geometry
problem = problems.problem4

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 0.8
ord = 3
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F,u):
    g = 9.81
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV + rho*g*u[1] - C


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-2)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((0,0))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0

obs = CF(0)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(2)+ grad(u)
    if form == "Functional":
        #LHS
        with TaskManager():
            BF += Variation(alpha * Gel_energy_functional(F,u).Compile()*dx)
            BF += delta * v[1]*dx
            BF += u[1]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
            #RHS
            BF += -(psik - psih)*v[1]*dx
            BF += -(obs + exp(psih))*vphi*dx
        return BF







In [ ]:
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 15

tol_newton = 1e-6
tol_QN = 1e-6  
tol_PG = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*20 ,softening_n))


# for num in tqdm(range(len(gammas)), desc="Gamma loop", chat_id="1366421620", token="7328845890:AAHPS3T9RbxDvlkFr1mDpaULNH98g7FMFKU"):
for num in range(len(gammas)):
    load = gammas[num]
    G.Set(load) #incremental softening
    if num == len(gammas)-1:
        tol_PG = 1e-10
        max_PG_it = 25
    for k in range(1,max_PG_it):

        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        if k != 0:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later
        print("Starting QN")
        for i in range(max_iter_qnewton):
            G_funs = u,v,alpha,delta,psih,psik 
            with TaskManager():
            BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            
            
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
            #iterations of newton for the NL problem at each QN step
            print("Solving nonlinear subproblem")
            for iter in range(max_iter_newton):
                with TaskManager():
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                gfu.vec.data += -w
                if abs(InnerProduct(w,res)) < tol_newton:
                    print("Tol achieved", InnerProduct(w,res))
                    break
            # check tolerance on QN
            qn_tol_val= Integrate((gfu.components[1])**2,mesh)
            if qn_tol_val < tol_QN:
                print("QN tol achieved",qn_tol_val )
                break
            # update psih
            psih.vec.data = psih.vec + gfu.components[1].vec
        # check error on gfu
        PG_error = Integrate((uk-gfu.components[0])**2,mesh)
        if PG_error < tol_PG:
            print("PG tol achieved",PG_error)
            break
    print("Error PG (not tol...)",PG_error)

    Draw(gfu.components[0],mesh)



Gamma loop:   0%|          | 0/15 [00:00<?, ?it/s]

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved -5.323710872393961e-07
Solving nonlinear subproblem
Tol achieved 7.032609180778478e-07
Solving nonlinear subproblem
Tol achieved -3.4536096273773013e-07
Solving nonlinear subproblem
Tol achieved -2.572986066109741e-07
Solving nonlinear subproblem
Tol achieved -2.8092068206793267e-07
Solving nonlinear subproblem
Tol achieved -3.856654973617851e-07
Solving nonlinear subproblem
Tol achieved -2.931602125703975e-07
QN tol achieved 2.0048925718326088e-08
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 5.927458526110552e-07
Solving nonlinear subproblem
Tol achieved 3.966819308755423e-07
Solving nonlinear subproblem
Tol achieved -5.453961801912516e-07
Solving nonlinear subproblem
Tol achieved -5.744455945315723e-07
Solving nonlinear subproblem
Tol achieved -5.132557615358143e-07
QN tol achieved 1.8700420873518117e-07
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 5.884162458209857e-07
Solving nonlinea

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 5.361287521307088e-07
Solving nonlinear subproblem
Tol achieved -3.387277991308452e-07
Solving nonlinear subproblem
Tol achieved -5.677910059789299e-07
Solving nonlinear subproblem
Tol achieved -7.146910864916505e-07
Solving nonlinear subproblem
Tol achieved 8.815655532190385e-07
Solving nonlinear subproblem
Tol achieved 4.055293507393375e-07
Solving nonlinear subproblem
Tol achieved 3.51655706645909e-07
Solving nonlinear subproblem
Tol achieved 2.241138287760739e-07
Solving nonlinear subproblem
Tol achieved 2.431862052407971e-07
Solving nonlinear subproblem
Tol achieved 2.1520806935904362e-07
Solving nonlinear subproblem
Tol achieved 1.365578919193235e-07
Solving nonlinear subproblem
Tol achieved 8.076636331295722e-08
Solving nonlinear subproblem
Tol achieved 5.5198309298485085e-08
Solving nonlinear subproblem
Tol achieved 3.888894967675183e-08
Solving nonlinear subproblem
Tol achieved 2.5509244874927845e-08
Solving nonlin

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.82425400716045e-07
Solving nonlinear subproblem
Tol achieved -8.553890177251141e-07
Solving nonlinear subproblem
Tol achieved -8.700634397324351e-07
Solving nonlinear subproblem
Tol achieved -5.007758731304023e-07
Solving nonlinear subproblem
Tol achieved 1.650520221061075e-07
Solving nonlinear subproblem
Tol achieved 5.6088269516393e-07
Solving nonlinear subproblem
Tol achieved 5.413111201906539e-07
Solving nonlinear subproblem
Tol achieved 3.897923684399507e-07
Solving nonlinear subproblem
Tol achieved 2.566766916886835e-07
Solving nonlinear subproblem
Tol achieved 1.6246533284842233e-07
Solving nonlinear subproblem
Tol achieved 9.983559633300743e-08
Solving nonlinear subproblem
Tol achieved 6.161214674847067e-08
Solving nonlinear subproblem
Tol achieved 3.9328578733285745e-08
Solving nonlinear subproblem
Tol achieved 2.575463883048255e-08
Solving nonlinear subproblem
Tol achieved 1.7036668630995165e-08
Solving nonlinea

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 6.775998186516477e-07
Solving nonlinear subproblem
Tol achieved -6.691492896154125e-07
Solving nonlinear subproblem
Tol achieved -3.317648208563767e-07
Solving nonlinear subproblem
Tol achieved -5.207546348244439e-07
Solving nonlinear subproblem
Tol achieved -4.1737868703333175e-07
Solving nonlinear subproblem
Tol achieved -1.2100027085574578e-07
Solving nonlinear subproblem
Tol achieved 2.497543507614234e-08
Solving nonlinear subproblem
Tol achieved 1.6139436203022382e-08
Solving nonlinear subproblem
Tol achieved -2.0084895470293175e-09
Solving nonlinear subproblem
Tol achieved 3.3352101749040245e-09
Solving nonlinear subproblem
Tol achieved 8.250442810714318e-09
Solving nonlinear subproblem
Tol achieved 6.131724281010808e-09
QN tol achieved 9.046982382588588e-07
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 6.280637808507742e-07
Solving nonlinear subproblem
Tol achieved -3.1437476591183494e-07
Solving non

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
PG it: 2
Starting QN
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
Solving nonlinear subproblem
S